In [42]:
import pandas as pd
import numpy as np

from datetime import datetime

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import shap

from scipy.stats import randint
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
import scikitplot as skplt
import sklearn as sk
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, Normalizer, PowerTransformer, OneHotEncoder
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as ltb
import xgboost as xgb

import statsmodels.api as sm
import statsmodels

from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from fredapi import Fred
import pandas_datareader as pdr

import warnings

In [43]:
sk.set_config(display='diagram')
warnings.filterwarnings("ignore")


In [44]:
fred = Fred(api_key='c0a3f23bdd23a65e6546b6d0e5f4d4a5')

# Load Data

In [45]:
#  Set start date
start_date = datetime.date(1940, 1, 1)
start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")

#  Federal Reserve Economic Data Service
data_source = 'fred'
unemployment_rate_code = 'UNRATE'
cpi_code = 'CORESTICKM157SFRBATL'
real_rate_code = 'REAINTRATREARAT1MO'

cpi_urban = 'CPIAUCSL'
inf_10y = 'T10YIEM'
ff_rate_code = 'EFFR'

#GDP GAP
p_gdp = 'GDPPOT'

TypeError: descriptor 'date' for 'datetime.datetime' objects doesn't apply to a 'int' object

In [ ]:
fred_df = pdr.DataReader(['FEDFUNDS','PCEPILFE','GDPC1','GDPPOT'], data_source, start_date)

# Analysis

In [ ]:
#t_df = df

plt.figure(figsize=(14,6))
mask = np.zeros_like(fred_df.corr())
mask[np.triu_indices_from(mask)] = True
_p = sns.heatmap(fred_df.corr().round(2), annot=True, mask=mask, cmap="plasma")

# Graph

In [ ]:
long_df = fred_df.copy()
long_df.reset_index(inplace=True)

long_df = long_df.melt(id_vars=['DATE'])


# Graph
g = sns.FacetGrid(long_df, col='variable', hue='variable',
                  height=3, aspect=6,col_wrap=1, sharey=False)

g.set_titles(col_template="{col_name}", row_template="{row_name}")
g.tick_params(axis='x', labelrotation=90) 
g.tight_layout()
g.map_dataframe(sns.lineplot, y="value", x="DATE")

plt.show()

# Taylor
The FRED® Blog
https://fredblog.stlouisfed.org/2014/04/the-taylor-rule/

This graph shows in blue the Taylor Rule, which is a simple formula that John Taylor devised to guide policymakers. It calculates what the federal funds rate should be, as a function of the output gap and current inflation. Here, we measure the output gap as the difference between potential output (published by the Congressional Budget Office) and real GDP. Inflation is measured by changes in the CPI, and we use a target inflation rate of 2%. We also assume a steady-state real interest rate of 2%. These are a lot of assumptions, and you are welcome to change them on the graph by playing around with the formula to see how the Taylor Rule matches up with the effective federal funds rate. To read up on the Taylor Rule, see the original article or an article by former St. Louis Fed president William Poole.

How this graph was created: To create a new series from several series, first add the series by modifying the existing series in the “Graph” tab. Once you have assembled them all, expand the series section in the same tab and “create your own transformation.” Finally, as the axis legend has become unwieldy, remove it by checking off the mark in the graph tab.

Suggested by: Christian Zimmermann

Update: A previous version did not multiply the output gap by 100.

View on FRED, series used in this post: FEDFUNDS, GDPC1, GDPDEF, GDPPOT

# Fred Calculations

In [ ]:
base_df = pd.read_csv('./data/fred.csv', parse_dates=['fund_date','taylor_date'])
base1_df = base_df[['fund_date','FEDFUNDS']].dropna().copy()
base1_df.sort_values(by='fund_date', inplace = True)
base2_df = base_df[['taylor_date','GDPPOT_GDPC1_GDPDEF_PC1']].copy()
base2_df.sort_values(by='taylor_date', inplace = True)

In [ ]:
# Graph
fig, ax = plt.subplots(figsize=(20,8))
ax.xaxis.grid()

# Plot
ax.plot(base2_df['taylor_date'], base2_df['GDPPOT_GDPC1_GDPDEF_PC1'], 'blue')
ax.plot(base1_df['fund_date'], base1_df['FEDFUNDS'], linestyle='--', color='red',linewidth=2)

plt.axhline(y=0, linestyle="dashed", color='lightgray')

ax.set(title='Fed Effective Funds Rate Prediction');

## Calculation 1

In [ ]:
gdpc1_df = pd.read_csv('./data/GDPC1.csv', parse_dates=['observation_date'], skiprows=10)
print('gdpc1_df:', gdpc1_df.shape)
gdppot_df = pd.read_csv('./data/GDPPOT.csv', parse_dates=['observation_date'], skiprows=10)
print('gdppot_df:', gdppot_df.shape)
gdpdef_df = pd.read_csv('./data/GDPDEF.csv', parse_dates=['observation_date'], skiprows=10)
print('gdpdef_df:', gdpdef_df.shape)
fed_df = pd.read_csv('./data/FEDFUNDS.csv', parse_dates=['observation_date'], skiprows=10)
print('fed_df:', fed_df.shape)

In [ ]:
taylor_df = gdpc1_df.merge(gdppot_df, how='outer', left_on='observation_date', right_on='observation_date')   \
                    .merge(gdpdef_df, how='outer', left_on='observation_date', right_on='observation_date')   \
                    .merge(fed_df, how='outer', left_on='observation_date', right_on='observation_date')
    
taylor_df.dropna(inplace=True)
taylor_df

In [ ]:
# Long 
long_df = taylor_df.melt(id_vars=['observation_date'])
print('long_df:', long_df.shape)

# Graph
fig, ax = plt.subplots(figsize=(20,8))
ax.xaxis.grid()

# Plot
#ax.plot(long_df['observation_date'], long_df['value'], color='variable')
sns.lineplot(x = 'observation_date', y = 'value', data=long_df, hue='variable')

ax.set(title='Fed Effective Funds Rate Prediction');

In [ ]:
#taylor_df = pd.DataFrame()
taylor_df['ffef_tr'] = taylor_df['GDPDEF_PC1'] + 2    \
                        +0.5*(taylor_df['GDPDEF_PC1'] - 2)    \
                        +0.5*(taylor_df['GDPC1'] - taylor_df['GDPPOT']) / taylor_df['GDPPOT'] * 100

taylor_df['gap_inf'] = 0.5*(taylor_df['GDPDEF_PC1'] - 2) 
taylor_df['gap_gdp'] = 0.5*(taylor_df['GDPC1'] - taylor_df['GDPPOT']) / taylor_df['GDPPOT'] * 100

taylor_df.tail(10)

In [ ]:
# Graph
fig, ax = plt.subplots(figsize=(20,8))
ax.xaxis.grid()

# Plot
ax.plot(taylor_df['observation_date'], taylor_df['ffef_tr'], 'blue')
ax.plot(taylor_df['observation_date'], taylor_df['FEDFUNDS'], linestyle='--', color='red',linewidth=2)

ax.set(title='Fed Effective Funds Rate Prediction');

In [ ]:
long_df = taylor_df[['observation_date','gap_inf','gap_gdp']].melt(id_vars=['observation_date'])

# Graph
fig, ax = plt.subplots(figsize=(20,8))
ax.xaxis.grid()

# Plot
sns.lineplot(data=long_df, x = 'observation_date', y = 'value', hue='variable')
ax.plot(taylor_df['observation_date'], taylor_df['FEDFUNDS'], linestyle='--', color='red',linewidth=2)

plt.axhline(y=0, linestyle="dashed", color='lightgray')

ax.set(title='Fed Effective Funds Rate Prediction');

# Taylor Rule

<b> Variables </b>
- nominal funds rate - FEDFUNDS
- real interest rate - REAINTRATREARAT1MO
- inflation - CPIAUCSL
- real gpd - real
- target = 0.02

#  Federal Reserve Economic Data Service
data_source = 'fred'
unemployment_rate_code = 'UNRATE'
cpi_code = 'CORESTICKM157SFRBATL'
real_rate_code = 'REAINTRATREARAT1MO'

cpi_urban = 'CPIAUCSL'
inf_10y = 'T10YIEM'
ff_rate_code = 'EFFR'

#GDP GAP
p_gdp = 'GDPPOT'

In [ ]:
t1_df = pdr.DataReader(['FEDFUNDS','REAINTRATREARAT1MO','CORESTICKM157SFRBATL','GDPC1','GDPPOT'], data_source, start_date)
#t1_df.dropna(inplace=True)
print('t1_df:', t1_df.shape)
t1_df.head(5)

In [ ]:
t2_df = pd.read_csv('./data/Holston_Laubach_Williams_real_time_estimates.csv', parse_dates=['Date'], skiprows=5)
t2_df.set_index('Date', inplace=True)
print('t2_df:', t2_df.shape)
t2_df.head(5)

In [ ]:
f1_df = t1_df.merge(t2_df, how='outer', left_index=True, right_index=True)
f1_df.dropna(inplace=True)
f1_df.index.rename('Date', inplace=True)
f1_df.head()

In [ ]:
target_inf = 2
alpha = 0.5
beta = 0.5

In [ ]:
f1_df['gap_gdp'] = alpha * (f1_df['GDPC1'] - f1_df['GDPPOT']) / f1_df['GDPPOT'] * 100
f1_df['gap_inf'] = beta * (f1_df['CORESTICKM157SFRBATL'] - target_inf)

f1_df['ffef_tr'] = f1_df['Real_Interest_Rate']  \
                        + f1_df['CORESTICKM157SFRBATL']  \
                        + f1_df['gap_gdp']  \
                        + f1_df['gap_inf']

f1_df.head()

In [ ]:
long_df = f1_df.copy()
long_df.reset_index(inplace=True)
long_df = long_df[['Date','FEDFUNDS','ffef_tr']].melt(id_vars=['Date'])

# Graph
fig, ax = plt.subplots(figsize=(20,8))
ax.xaxis.grid()

# Plot
sns.lineplot(data=long_df, x = 'Date', y = 'value', hue='variable')
#ax.plot(taylor_df['observation_date'], taylor_df['FEDFUNDS'], linestyle='--', color='red',linewidth=2)

ax.set(title='Fed Effective Funds Rate Prediction');

In [ ]:
long_df = f1_df.copy()
long_df.reset_index(inplace=True)
long_df = long_df[['Date','gap_gdp','Output_Gap']].melt(id_vars=['Date'])

# Graph
fig, ax = plt.subplots(figsize=(20,8))
ax.xaxis.grid()

# Plot
sns.lineplot(data=long_df, x = 'Date', y = 'value', hue='variable')
#ax.plot(taylor_df['observation_date'], taylor_df['FEDFUNDS'], linestyle='--', color='red',linewidth=2)

ax.set(title='Fed Effective Funds Rate Prediction');

## Taylor 2

In [ ]:
p_smooth = 0.85

p_inf = 0.5
target_inf = 2

p_unemp = 0.5
target_unemp = 5

In [ ]:
t0_df = pdr.DataReader(['DFEDTARU','DFEDTARL','DFEDTAR'], data_source, start_date)
t0_df['DFEDTAR'] = np.where(t0_df['DFEDTAR'].isnull(), t0_df[['DFEDTARU', 'DFEDTARL']].mean(axis=1), t0_df['DFEDTAR'])
t0_df = t0_df[['DFEDTAR']]
print('t0_df:', t0_df.shape)
t0_df.head(5)

In [ ]:
t1_df = pdr.DataReader(['FEDFUNDS','CORESTICKM157SFRBATL','UNRATE'], data_source, start_date)
#t1_df.dropna(inplace=True)
print('t1_df:', t1_df.shape)
t1_df.head(5)

In [ ]:
t2_df = pd.read_csv('./data/Holston_Laubach_Williams_real_time_estimates.csv', parse_dates=['Date'], skiprows=5)
t2_df.set_index('Date', inplace=True)
print('t2_df:', t2_df.shape)
t2_df.head(5)

In [ ]:
f2_df = t1_df.merge(t2_df, how='outer', left_index=True, right_index=True)   \
                .merge(t0_df, how='outer', left_index=True, right_index=True)

f2_df.dropna(inplace=True)
f2_df.index.rename('Date', inplace=True)
f2_df['DFEDTAR_t-1'] = f2_df['DFEDTAR'].shift(periods=1)
f2_df.dropna(inplace=True)
f2_df.head()

In [ ]:
f2_df['gap_unemp'] = p_unemp * (f1_df['GDPC1'] - f2_df['GDPPOT']) / f2_df['GDPPOT'] * 100
f2_df['gap_inf'] = p_inf * (f2_df['CORESTICKM157SFRBATL'] - target_inf)


f2_df['ffef_tr'] = p_smooth * f2_df['DFEDTAR_t-1']    \
                    + (1 - p_smooth)    \
                    * f2_df['Real_Interest_Rate'] 


f2_df['ffef_tr'] = f2_df['Real_Interest_Rate']  \
                        + f1_df['CORESTICKM157SFRBATL']  \
                        + f1_df['gap_gdp']  \
                        + f1_df['gap_inf']

f2_df.head()

In [ ]:
t01_df = pdr.DataReader(['DFEDTARU','DFEDTARL','DFEDTAR'], data_source, start_date)
t01_df['DFEDTAR'] = np.where(t01_df['DFEDTAR'].isnull(), t01_df[['DFEDTARU', 'DFEDTARL']].mean(axis=1), t01_df['DFEDTAR'])
t01_df = t01_df[['DFEDTAR']]
t01_df.tail(30)